# Кластеризация, метод DBSCAN  

Кластеризация (англ. cluster analysis) — задача группировки множества объектов на подмножества (кластеры) таким образом, чтобы объекты из одного кластера были более похожи друг на друга, чем на объекты из других кластеров по какому-либо критерию. 

Основанная на плотности пространственная кластеризация для приложений с шумами (англ. Density-based spatial clustering of applications with noise, DBSCAN) — это алгоритм кластеризации данных, который предложили Маритин Эстер, Ганс-Петер Кригель, Ёрг Сандер и Сяовэй Су в 1996. Это алгоритм кластеризации, основанной на плотности — если дан набор точек в некотором пространстве, алгоритм группирует вместе точки, которые тесно расположены (точки со многими близкими соседями), помечая как выбросы точки, которые находятся одиноко в областях с малой плотностью (ближайшие соседи которых лежат далеко). DBSCAN является одним из наиболее часто используемых алгоритмов кластеризации, и наиболее часто упоминается в научной литературе. 

## Шаги алгоритма: 


Алгоритм начинается с произвольной точки, которая не была посещена, находится количество соседей. 

Если количетсво соседей содержит >= минимального количества точек в указанной окрестности (eps) начинается формирование кластера. В противном случае точка помечается как шум. Эта точка может быть позже найдена в eps-окрестности другой точки и, таким образом, может стать частью кластера. 

Если точка найдена как центральная точка, то точки в окрестности также являются частью кластера. Таким образом, все точки, найденные в eps окрестности, добавляются вместе с их собственной eps окрестностью, если они также являются центральными точками. 

Вышеописанный процесс продолжается до тех пор, пока кластер, связанный плотностью, не будет найден полностью. 

Процесс возобновляется с новой точкой, которая может быть частью нового кластера или помечена как шум. 

Рассмотрим написанный код на Python. 

Импортируем небодимые библиотеки и создадим классы. У нас будет класс Point(Точка), а также Enum View, который указывает, не просмотрена ли точка, и является ли она шумом. Библиотека math понадобится для вычисления расстояния между точками, чтобы узнать, лежат ли они в допустимой окрестности.

In [ ]:
import enum
import math


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class View(enum.Enum):
    UNDEFINED = 0
    NOISE = 1

Ниже происходит заполнение входных данных, на которых будет работать алгоритм. Необходимо получить окрестность (eps), Минимальное количество точек (minPoints) и непосредственно сами точки. Затем происходит вызов функции DBSCAN(), и вывод результата.

In [ ]:
points = list()
eps = float(input("Окрестность: ",))
minPoints = int(input("Минимальное количество точек: "))
n = int(input("Количество точек: ",))
for i in range(n):
    print(i, ":")
    x = float(input("Координата х точки: ",))
    y = float(input("Координата y точки: ",))
    points.insert(i, Point(x, y))
result = DBSCAN()
for item in result.items():
    print("Point: (", item[0].x,", ", item[0].y,"); Claster: ", item[1])

## Рассмотрим функцию DBSCAN().

В переменной clusters хранится текущий индекс кластера. Заведем словарь result, где ключем будет точка, а значением - номер кластера (или пометка о шуме). Изначально все точки не просмотрены:

In [ ]:
def DBSCAN():
    clusters = 0
    result = {}
    for j in range(len(points)):
        result[points[j]] = View.UNDEFINED

На этом этапе мы ищем ядра кластеров. Такие точки, которые принадежат кластеру, но не являются крайними. Для каждой непросмотренной точки найдем количество смежных точек (включая исходную). Если количество этих точек меньше minPoints, то точка является шумом (или в последующем крайней точкой), можно рассматривать следующую точку.

In [ ]:
    for j in range(len(points)):
        if result[points[j]] != View.UNDEFINED:
            continue
        neighbors = find_neighbors(points[j])
        if len(neighbors) < minPoints:
            result[points[j]] = View.NOISE
            continue

В противном случае, мы нашли ядро кластера. А значит, все соседние точки также принадлежат этому кластеру. Если точка является соседом и была помечена шумом, значит, она принадлежит этому кластеру и является крайней. Если же точка уже принадлежит какому-либо кластеру, значит, можно ее не рассматривать и переходить к слеующей. Если точку вообще не рассматривали (но при этом она уже является соседом), то добавляем ее в кластер, находим соседние точки, и если количество смежных точек (включая исходную) больше или равна minPoints, то эта точка также является ядром кластера, а соседние точки, которые отправляются в list на дальнейшее рассмотрение, также принадлежат этому кластеру. Таким образом, мы просмотрим все смежные точки, выйдем из цикла и повторим данную процедуру для всех не рассмотренных точек, находя новые кластеры или же помечая их шумом. В конце метода вернем словать result, где для каждой точки будет информация либо по номеру кластера, либо по шуму.

In [ ]:
        clusters += 1
        result[points[j]] = clusters
        s = neighbors.copy()
        s.remove(points[j])

        k = 0
        while k < len(s):
            if result[s[k]] == View.NOISE:
                result[s[k]] = clusters
            if result[s[k]] != View.UNDEFINED:
                k += 1
                continue
            result[s[k]] = clusters
            neighbors = find_neighbors(s[k])
            if len(neighbors) >= minPoints:
                for neighbor in neighbors:
                    s.insert(len(s), neighbor)
            k += 1

    return result

Функция find_neighbors(), которая ищет косяк точек, смежной к исходной, не дальше расстояния eps:

In [ ]:
neighbors = list()
    for point in points:
        dist = math.sqrt((item.x - point.x) ** 2 + (item.y - point.y) ** 2)
        if dist <= eps:
            neighbors.insert(len(neighbors), point)
    return neighbors

## Примеры

### Пример 1

Рассмотрим случай со следущими исходными данными:<br/>
Окрестность: 1<br/>
Минимальное количество точек: 2<br/>
Точки: (0, 0); (0, 1); (1, 0); (1, 1)<br/>
Вывод программы:

In [ ]:
Point: ( 0.0 ,  0.0 ); Claster:  1
Point: ( 1.0 ,  0.0 ); Claster:  1
Point: ( 0.0 ,  1.0 ); Claster:  1
Point: ( 1.0 ,  1.0 ); Claster:  1

Действительно, все точки буду принадлежать одному кластеру. Все четыре точки имеют по два соседа, там образом, количество смежных точек равно трем, что больше minPoints. В этом примере все точки являются ядрами первго кластера.

### Пример 2

Рассмотрим случай со следущими исходными данными:<br/>
Окрестность: 1<br/>
Минимальное количество точек: 4<br/>
Точки: (1.5, 0.0); (2.5, 0.0); (2.6, 0.0); (2.7, 0.0); (2.8, 0.0); (3.8, 0.0); (4.0 , 0.0)<br/>
Вывод программы:

In [ ]:
Point: ( 1.5 ,  0.0 ); Claster:  1
Point: ( 2.5 ,  0.0 ); Claster:  1
Point: ( 2.6 ,  0.0 ); Claster:  1
Point: ( 2.7 ,  0.0 ); Claster:  1
Point: ( 2.8 ,  0.0 ); Claster:  1
Point: ( 3.8 ,  0.0 ); Claster:  1
Point: ( 4.0 ,  0.0 ); Claster:  View.NOISE

Действительно, точка с координатами (1.5, 0.0) не имеет трех соседей (для minPoints = 4), однако она соседствует с точкой с координатами (2.5, 0.0), у которой есть три соседа, а значит, она принадлежит кластеру. Такая же ситуация с точкой с координатами (3.8, 0.0). Однако точка с координатами (4.0 , 0.0) соседствует только с крайней точкой, которая в свою очередь имеет только двух соседей, а значит не является ядром, и соседние с ней точки принадлежат кластеру только в случае наличя необходимого количества соседей, что с точкой (4.0 , 0.0) не выполняется. Таким образом, кластеру она не принадлежит и помечена как шум View.NOISE.